In [1]:
#setup openrouteservice, tkiter, folium 
import tkinter as tk
import openrouteservice as ors
import folium
from openrouteservice import optimization

# API key ORS
client = ors.Client(key='eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjI4OTNlZjlmMWU2MTQ5MjBiN2FiYzdkOGYyMWFlOGI0IiwiaCI6Im11cm11cjY0In0=')

In [24]:
def main():
    coords = []
    vehicle_start = None

    def format_coords():
        lines = []
        for lon, lat in coords:
            # keep 8 decimal places like screenshot
            lines.append(f"  [{lon:.8f}, {lat:.8f}],")
        if lines:
            body = "\n".join(lines)
            return f"coords = [\n{body}\n]"
        else:
            return "coords = [\n]"

    def update_display():
        text = format_coords()
        output_text.config(state="normal")
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, text)
        output_text.config(state="disabled")
        count_label.config(text=f"Count: {len(coords)}")

    def add_coordinate():
        lon_s = longitude_entry.get().strip()
        lat_s = latitude_entry.get().strip()
        try:
            lon = float(lon_s)
            lat = float(lat_s)
        except ValueError:
            status_label.config(text="Invalid number", fg="red")
            return
        coords.append((lon, lat))
        longitude_entry.delete(0, tk.END)
        latitude_entry.delete(0, tk.END)
        status_label.config(text=f"Last added: [{lon:.8f}, {lat:.8f}]", fg="black")
        update_display()
        
    def addtostart():
        nonlocal vehicle_start
        lon_st = longitude_entry2.get().strip()
        lat_st = latitude_entry2.get().strip()
        try:
            lon2 = float(lon_st)
            lat2 = float(lat_st)
        except ValueError:
            status_label.config(text="Invalid number", fg="red")
            return None
        vehicle_start = (lon2, lat2)
        longitude_entry2.delete(0, tk.END)
        latitude_entry2.delete(0, tk.END)
        status_label.config(text=f"Last added to start: [{lon2:.8f}, {lat2:.8f}]", fg="black")
        update_display()
        return vehicle_start

    def copy_to_clipboard():
        text = format_coords()
        root.clipboard_clear()
        root.clipboard_append(text)
        status_label.config(text="Copied to clipboard", fg="green")

    def clear_all():
        coords.clear()
        update_display()
        status_label.config(text="Cleared", fg="black")

    root = tk.Tk()
    root.title("Quick Coords Collector")
    ## Bagian input longitude dan latitude
    input_frame = tk.LabelFrame(root, text="Add Coordinate")
    input_frame.grid(row=0, column=0, padx=10, pady=10, sticky="w")

    tk.Label(input_frame, text="Longitude").grid(row=0, column=0)
    tk.Label(input_frame, text="Latitude").grid(row=0, column=1)

    longitude_entry = tk.Entry(input_frame, width=18)
    latitude_entry = tk.Entry(input_frame, width=18)
    longitude_entry.grid(row=1, column=0, padx=5, pady=5)
    latitude_entry.grid(row=1, column=1, padx=5, pady=5)

    add_btn = tk.Button(input_frame, text="Add", command=add_coordinate, width=10)
    add_btn.grid(row=1, column=2, padx=5)

    controls_frame = tk.Frame(root)
    controls_frame.grid(row=1, column=0, sticky="w", padx=10)

    copy_btn = tk.Button(controls_frame, text="Copy to clipboard", command=copy_to_clipboard)
    copy_btn.grid(row=0, column=0, padx=(0,10))

    clear_btn = tk.Button(controls_frame, text="Clear", command=clear_all)
    clear_btn.grid(row=0, column=1)

    count_label = tk.Label(controls_frame, text="Count: 0")
    count_label.grid(row=0, column=2, padx=(10,0))

    status_label = tk.Label(root, text="", anchor="w")
    status_label.grid(row=3, column=0, sticky="w", padx=10, pady=(4,0))

    ## bagian output
    output_frame = tk.LabelFrame(root, text="Output")
    output_frame.grid(row=2, column=0, padx=10, pady=10, sticky="we")

    output_text = tk.Text(output_frame, height=10, width=60, wrap="none")
    output_text.pack(fill="both", expand=True)
    output_text.config(state="disabled")

    # bagian posisi berangkat
    input_frame2 = tk.LabelFrame(root, text="Posisi Awal")
    input_frame2.grid(row=3, column=0, padx=10, pady=10, sticky="w")

    tk.Label(input_frame2, text="Longitude").grid(row=0, column=0)
    tk.Label(input_frame2, text="Latitude").grid(row=0, column=1)

    longitude_entry2 = tk.Entry(input_frame2, width=18)
    latitude_entry2 = tk.Entry(input_frame2, width=18)
    longitude_entry2.grid(row=4, column=0, padx=5, pady=5)
    latitude_entry2.grid(row=4, column=1, padx=5, pady=5)

    add_btn2 = tk.Button(input_frame2, text="Set", command=addtostart, width=10)
    add_btn2.grid(row=4, column=2, padx=5)

    
    
    # keyboard bindings for quicker usage
    root.bind('<Return>', lambda e: add_coordinate())

    # start with focus at longitude
    longitude_entry.focus()

    root.mainloop()
    
    # Return coords array after GUI is closed
    return coords, vehicle_start

if __name__ == '__main__':
    coords, vehicle_start = main()


In [25]:
#visualisasi koordinat dengan memberi marking pada map
m = folium.Map(location=list(reversed([104.7610533, -2.9944282])), tiles='cartodbpositron', zoom_start=13) # pusat dari map

for coord in coords:
    folium.Marker(location=list(reversed(coord))).add_to(m) # marker untuk gardu

folium.Marker(location=list(reversed(vehicle_start)), icon = folium.Icon(color="red")).add_to(m) # koordinat titik berangkat
m

## pendaftaran vehicle 
vehicles = [
    ors.optimization.Vehicle(id=0, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[5]),
    ors.optimization.Vehicle(id=1, profile='driving-car', start=vehicle_start, end=vehicle_start, capacity=[5])
]

#jobs
jobs  = [ors.optimization.Job(id=index,  location = coords, amount=[1]) for index, coords  in enumerate(coords)] ## jobs v1
#jobs =  [ors.optimization.Job(id= index, **job) for index,  job in enumerate(coords)]
optimized = client.optimization(jobs=jobs,  vehicles=vehicles, geometry=True) #sudah dioptimisasi

line_colors=['green', 'orange', 'blue', 'yellow']

for route in optimized['routes']:
    folium.PolyLine(locations=[list(reversed(coords)) for  coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],  color=line_colors[route['vehicle']]).add_to(m)

m.save("peta_rute.html")